In [1]:
#!/usr/bin/env python3

import sys
import pandas as pd
import datetime
import requests
import numpy as np
from pandas import ExcelWriter
import os
import glob
import pytz
from dateutil.tz import gettz
import timeit
from time import sleep

# from datetime import datetime
from datetime import timedelta
import time
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)

In [2]:
phase='A'
mapping = {}
mapping['p'] = 'pwr%s' % phase
mapping['q'] = 'rpwr%s' % phase
mapping['v'] = 'vlt%s' % phase
mapping['i'] = 'cur%s' % phase
mapping['s'] = 'apwr%s' % phase
mapping['f'] = 'freq%s' % phase
mapping['pF'] = 'cos%s' % phase
mapping['cF'] = 'scre%s' % phase
mapping['an'] = 'angle%s' % phase
mapping['cE'] = 'cnrg%s' % phase
mapping['pE'] = 'pnrg%s' % phase
mapping['cR'] = 'crnrg%s' % phase
mapping['pR'] = 'prnrg%s' % phase
mapping['avgSP'] = 'avgSP%s' %phase
mapping['sign'] = 'sign%s' %phase

In [3]:
import ast
file1 = open('myfile.txt', 'r') 
Lines = file1.readlines() 

df = pd.DataFrame([])
# Strips the newline character 
for line in Lines:
#     print(line.rstrip())
    t = line.rstrip()#.split('&')
    t = t.replace(',', '.')
    t = t.split('&')
    if t[0] != '':
        if t[2][0] == 'L':
            t[2] = 'L=' + t[2][1]
        else:
            t[2] = 'P=' + t[2][1]
            t[3] = 'S=' + t[3][1]
    
        mydict = dict(zip((v.split('=')[0] for v in t), ((v.split('=')[1]) for v in t)))
#         mydict = ast.literal_eval(line.strip())
        df = pd.concat([df,pd.DataFrame([mydict])])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
for col in df.columns:
    if ((col=='utc') or (col=='sN')):
        df[col] = df[col].astype('int')
    else:
        df[col] = df[col].astype('float')
df = df.loc[df['p']>=0]
df = df[['p','q','sN','utc','ms']]
df.reset_index(inplace= True, drop=True)

df.head()

,p,q,sN,utc,ms
0,245.6,110.4,5770705,656764117,674057490.0
1,245.8,112.0,5770706,656764117,674057510.0
2,245.9,111.1,5770707,656764117,674057530.0
3,245.8,110.8,5770708,656764117,674057550.0
4,245.6,111.1,5770709,656764117,674057570.0


In [8]:
file2 = open('myfile2.txt', 'r') 
Lines = file2.readlines() 

df2 = pd.DataFrame([])
# Strips the newline character 
for line in Lines:
#     print(line.rstrip())
    t = line.rstrip()#.split('&')
    t = t.replace(',', '.')
    t = t.split('&')
    if t[0] != '':
        if t[2][0] == 'L':
            t[2] = 'L=' + t[2][1]
        else:
            t[2] = 'P=' + t[2][1]
            t[3] = 'S=' + t[3][1]
    
        mydict = dict(zip((v.split('=')[0] for v in t), ((v.split('=')[1]) for v in t)))
#         mydict = ast.literal_eval(line.strip())
        df2 = pd.concat([df2,pd.DataFrame([mydict])])
df2

,sN,ms,P,S,avgP,minP,maxP,stdP,avgR,minR,maxR,stdR
0,5770940,674062159,1,1,2141.3,2129.1,2154.3,153.0,2147.8,2136.0,2158.1,35.7
0,5770943,674062209,1,2,2122.1,2113.1,2128.6,125.5,2127.1,2119.8,2135.3,141.5
0,5770947,674062259,1,3,2103.1,2096.3,2112.3,115.9,2108.6,2101.5,2119.1,145.6
0,5770950,674062309,1,4,2088.0,2082.3,2095.8,157.8,2093.1,2086.8,2101.1,86.5


In [9]:
for col in df2.columns:
    if ((col=='utc') or (col=='sN')):
        df2[col] = df2[col].astype('int')
    else:
        df2[col] = df2[col].astype('float')
df2.head()

,sN,ms,P,S,avgP,minP,maxP,stdP,avgR,minR,maxR,stdR
0,5770940,674062159.0,1.0,1.0,2141.3,2129.1,2154.3,153.0,2147.8,2136.0,2158.1,35.7
0,5770943,674062209.0,1.0,2.0,2122.1,2113.1,2128.6,125.5,2127.1,2119.8,2135.3,141.5
0,5770947,674062259.0,1.0,3.0,2103.1,2096.3,2112.3,115.9,2108.6,2101.5,2119.1,145.6
0,5770950,674062309.0,1.0,4.0,2088.0,2082.3,2095.8,157.8,2093.1,2086.8,2101.1,86.5


In [10]:
df2.reset_index(inplace= True, drop=True)
df2.head()

,sN,ms,P,S,avgP,minP,maxP,stdP,avgR,minR,maxR,stdR
0,5770940,674062159.0,1.0,1.0,2141.3,2129.1,2154.3,153.0,2147.8,2136.0,2158.1,35.7
1,5770943,674062209.0,1.0,2.0,2122.1,2113.1,2128.6,125.5,2127.1,2119.8,2135.3,141.5
2,5770947,674062259.0,1.0,3.0,2103.1,2096.3,2112.3,115.9,2108.6,2101.5,2119.1,145.6
3,5770950,674062309.0,1.0,4.0,2088.0,2082.3,2095.8,157.8,2093.1,2086.8,2101.1,86.5


In [13]:
df['diff'] = df['sN'] - df['sN'].shift()
df = df.dropna()

In [15]:
# 
df

,p,q,sN,utc,ms,diff
1,245.8,112.0,5770706,656764117,674057510.0,1.0
2,245.9,111.1,5770707,656764117,674057530.0,1.0
3,245.8,110.8,5770708,656764117,674057550.0,1.0
4,245.6,111.1,5770709,656764117,674057570.0,1.0
5,245.4,109.5,5770710,656764117,674057590.0,1.0
6,245.1,109.5,5770711,656764117,674057610.0,1.0
7,245.1,110.3,5770712,656764117,674057630.0,1.0
8,245.1,110.6,5770713,656764117,674057650.0,1.0
9,245.3,109.4,5770714,656764117,674057670.0,1.0
10,245.5,110.3,5770715,656764117,674057690.0,1.0


In [18]:
128-85

43

In [16]:
df['p'].iloc[40:60].mean()

248.31

In [17]:
128-85

43

In [59]:
df['p'].iloc[72:122].mean()

0.56

In [57]:
np.abs(float(df['p'].iloc[-60:-10].mean()) - 0)

4.127999999999999

In [55]:
df2

,sN,ms,P,S,avgP,minP,maxP,stdP,avgR,minR,maxR,stdR
0,8169,458463.0,1.0,1.0,3.9,3.9,4.0,163.3,8.3,8.3,8.3,8.3
1,8175,458513.0,1.0,2.0,3.9,3.9,4.1,163.3,8.1,8.1,8.3,8.3
2,8181,458564.0,1.0,3.0,4.0,4.0,4.3,163.3,8.1,8.1,8.3,8.1
3,8187,458614.0,1.0,4.0,4.0,4.0,4.1,163.3,8.1,8.1,8.3,8.3


In [42]:
from datetime import datetime
ts = '656677837'
int((datetime(2000, 1, 1) - datetime(1970, 1, 1)).total_seconds()) + int(ts)

1603362637